In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Possible GPUS

In [2]:
import torch
import torchunlearn

In [3]:
torchunlearn.__version__

'0.1'

In [6]:
PATH = "./models/"
NAME = "Test"
SAVE_PATH = PATH + NAME

MODEL_NAME = "ResNet18"
DATA_NAME = "CIFAR10"
MEAN = [0.4914, 0.4822, 0.4465]
STD = [0.2023, 0.1994, 0.2010]
BATCH_SIZE = 128
N_CLASSES = 10

N_VALIDATION = 1000

PRETRAINED_PATH = PATH + f"{DATA_NAME}_Standard/last.pth"

## Prepare Datasets

In [7]:
from torchunlearn.utils.data import UnlearnDataSetup, MergedLoaders

In [8]:
setup = UnlearnDataSetup(data_name=DATA_NAME, n_classes=N_CLASSES, mean=MEAN, std=STD)

#### For Random-forgetting

In [11]:
train_loaders, test_loaders = setup.get_loaders_for_rand(batch_size=BATCH_SIZE, ratio=0.1, stratified=True,
                                                         train_shuffle_and_transform=False, drop_last_train=True, seed=42)

Files already downloaded and verified
Files already downloaded and verified
Data Loaded!
Train Data Length : 50000
Test Data Length : 10000
Files already downloaded and verified
Files already downloaded and verified
Data Loaded (w/ Validation Set)!
Train Data Length : 45000
Val Data Length : 5000
Test Data Length : 10000


#### For Classwise-forgetting

In [12]:
train_loaders, test_loaders = setup.get_loaders_for_classwise(batch_size=BATCH_SIZE, omit_label=1,
                                                              train_shuffle_and_transform=True, drop_last_train=True)

Files already downloaded and verified
Files already downloaded and verified
Data Loaded! (w/ Label Filtering)
Train Data Length : 45000
Test Data Length : 9000
Files already downloaded and verified
Files already downloaded and verified
Data Loaded! (w/ Label Filtering)
Train Data Length : 5000
Test Data Length : 1000


## Prepare Model

In [15]:
model = torchunlearn.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
rmodel = torchunlearn.RobModel(model, n_classes=N_CLASSES, normalization_used={'mean':MEAN, 'std':STD}).cuda()
# rmodel = rmodel.set_parallel()
rmodel.load_dict(PRETRAINED_PATH)

ResNet18 is loaded.
Model loaded.
Record Info:
OrderedDict([('Epoch', 200), ('Iter', 390), ('Cost', 0.008486972190439701), ('lr', 0.0010000000000000002), ('s/it', 0.011221407692307691)])


## Start Unlearn

In [28]:
from torchunlearn.unlearn.trainers.finetune import Finetune
trainer = Finetune(rmodel)

In [29]:
EPOCH = 5

In [30]:
merged_train_loader = MergedLoaders(train_loaders)

In [32]:
merged_train_loader = MergedLoaders(train_loaders)
trainer.setup(optimizer=f"SGD(lr=0.01, momentum=0.9, weight_decay=5e-4)",
              scheduler=None, scheduler_type=None,
              minimizer=None, n_epochs=EPOCH,
             )

In [33]:
loaders_with_flags = {
    "(R)": train_loaders['Retain'],
    "(F)": train_loaders['Forget'],
    "(Te)": test_loaders['Test'],
}

trainer.record_rob(loaders_with_flags, n_limit=N_VALIDATION)

In [34]:
trainer.fit(train_loaders=merged_train_loader, n_epochs=EPOCH,
            save_path=SAVE_PATH, save_best={"Clean(R)":"HB", "Clean(F)":"LBO"},
            save_type=None, save_overwrite=True, record_type="Epoch")

[Finetune]
Training Information.
-Epochs: 5
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
-Scheduler: None
-Minmizer: None
-Save Path: ./models/Test
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
---------------------------------------------------------------------
Epoch   Cost     Clean(R)   Clean(F)   Clean(Te)   lr       s/it     
1       0.0913   96.4844    91.0156    91.7969     0.0100   0.0572   
---------------------------------------------------------------------
2       0.0524   96.3867    68.6523    91.0156     0.0100   0.0541   
---------------------------------------------------------------------
3       0.0884   95.8008    54.3945    91.0156     0.0100   0.0521   
---------------------------------------------------------------------
4       0.0525   96.5820    45.0195    90.0391     0.0100   0.0519   
---------------------------------------------------------------------
5       0.107